In [2]:

import requests
from time import sleep
import pandas as pd
import unicodedata
import string
import urllib
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#③実行する
import json
#ServiceAccountCredentials：Googleの各サービスへアクセスできるservice変数を生成します。
# from oauth2client.service_account import ServiceAccountCredentials 
# from gspread_dataframe import get_as_dataframe, set_with_dataframe
#2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
import ssl
from datetime import datetime as dt
# from utils import *
import datetime
import time
import os

def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text

def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 


def read_spreadsheet(workbook,worksheet):
    read_worksheet_df = pd.DataFrame(worksheet.get_all_values())
    read_worksheet_df.columns = list(read_worksheet_df.loc[0, :])
    read_worksheet_df.drop(0, inplace=True)
    return read_worksheet_df

def write_dataframe_to_spreadsheet(write_spreadsheet_df,worksheet:str,title_text:str):
    workbook.add_worksheet(title=title_text, rows=len(write_spreadsheet_df), cols=len(write_spreadsheet_df.columns))
    worksheet = workbook.worksheet(title_text)
    #df_a['G数'] = df_a['G数'].map(set_round)'
    set_with_dataframe(worksheet, write_spreadsheet_df)

In [29]:
2524/2

1262.0

In [32]:
dmm_app_tenpo_name_list_df[:int(2524/2)]

,ホール名,大当たり情報契約
0,マルハンメガシティ三島駿東,ANJ（P'sCUBE)
1,マルハンメガシティ長野,ANJ（P'sCUBE)
2,アムズツインパーク,ANJ（P'sCUBE)
3,キコーナ松戸店,プロテラス （テラスモバイル）
4,キコーナ新宿三丁目店,プロテラス （テラスモバイル）
...,...,...
1257,タイホー御殿場大坂,ANJ（P'sCUBE)
1258,タイホー御殿場ぐみ沢,ANJ（P'sCUBE)
1259,スリーエー1・2,ANJ（P'sCUBE)
1260,タイホー浜松三幸町,ANJ（P'sCUBE)


In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
anaslo_scraping_error_tenpo_name_dict = {}
browser = webdriver.Chrome(ChromeDriverManager().install())
serched_df_3 = pd.DataFrame(index=[], columns=[])
dmm_and_anaslo_tenpo_name_dict = {}
#no_scraping_anaslo_tenpo_name = []
i = 0
for i,row in dmm_app_tenpo_name_list_df[2000:].iterrows():
    try:
        dmm_tenpo_name = row['ホール名']
        url = f"https://www.google.com/search?q=アナスロ  {dmm_tenpo_name} データ一覧&hl=ja&source=hp/"
        print(i,dmm_tenpo_name)
        #print('データ取得中...30秒ほど時間がかかります。')
        browser.get(url)
        html = browser.page_source.encode('utf-8')
        elems_rec = browser.find_elements_by_class_name('MjjYud')
        elems_rec[0].text.split('\n')
        #print('アナスロと出る部分',elems_rec[0].text.split('\n')[1])

        scraping_anaslo_name = elems_rec[0].text.split('\n')[0]
        dmm_and_anaslo_tenpo_name_dict[dmm_tenpo_name] = scraping_anaslo_name
        serched_df_3 = pd.concat([serched_df_3,pd.DataFrame([[dmm_tenpo_name,row['大当たり情報契約'],scraping_anaslo_name]])],axis=0)
        time.sleep(2)
    except Exception as e:
        anaslo_scraping_error_tenpo_name_dict[dmm_tenpo_name] =  e
        print('エラー店舗',dmm_tenpo_name,e)
        continue
    #     break
#data_df.to_csv(csv_file_name)

In [41]:
anaslo_scraping_error_tenpo_name_dict

{}

In [52]:
concat_serch_df = pd.concat([serched_df_1,serched_df_2,serched_df_3],axis=0)
concat_serch_df

,DMM店舗名(変換前),取得ドメイン,アナスロ(変換後)
0,マルハンメガシティ三島駿東,ANJ（P'sCUBE),マルハンメガシティ三島駿東 データ一覧
1,マルハンメガシティ長野,ANJ（P'sCUBE),マルハンメガシティ長野 データ一覧
2,アムズツインパーク,ANJ（P'sCUBE),アムズツインパーク データ一覧
3,キコーナ松戸店,プロテラス （テラスモバイル）,キコーナ松戸店 データ一覧
4,キコーナ新宿三丁目店,プロテラス （テラスモバイル）,キコーナ新宿三丁目店 データ一覧
...,...,...,...
0,エアポート777土岐店,グローリーナスカ（PAPIMO-NET),エアポート777土岐店 データ一覧
0,747三和店,ANJ（P'sCUBE),747三和店 データ一覧
0,RITZ高槻店,ANJ（P'sCUBE),RITZ高槻店 データ一覧
0,RITZ防府店,ANJ（P'sCUBE),RITZ防府店 データ一覧


In [55]:
concat_serch_df[~concat_serch_df['アナスロ(変換後)'].str.contains('データ一覧')]

,DMM店舗名(変換前),取得ドメイン,アナスロ(変換後)
49,キコーナ香里園店,ANJ（P'sCUBE),大阪府
56,ガイア瑞江店,ANJ（P'sCUBE),アナスロ - 全国パチスロデータ・出玉情報
57,ガイアネクスト三ツ境Ⅰ,ANJ（P'sCUBE),神奈川県
58,ガイア西船橋店,ANJ（P'sCUBE),アナスロ - 全国パチスロデータ・出玉情報
59,メガガイア青森中央,ANJ（P'sCUBE),青森県
...,...,...,...
0,スーパーハイペリオンセブン,ANJ（P'sCUBE),アナスロ - 全国パチスロデータ・出玉情報
0,パチンコ&スロット タイガー川棚店,プレネット（P=DO),アナスロ - 全国パチスロデータ・出玉情報
0,パチンコ&スロット サンタイガー店,プレネット（P=DO),アナスロ - 全国パチスロデータ・出玉情報
0,大学院,ANJ（P'sCUBE),2021/11/05 大学院 データまとめ


In [56]:
concat_serch_df[concat_serch_df['アナスロ(変換後)'].str.contains('データ一覧')]

,DMM店舗名(変換前),取得ドメイン,アナスロ(変換後)
0,マルハンメガシティ三島駿東,ANJ（P'sCUBE),マルハンメガシティ三島駿東 データ一覧
1,マルハンメガシティ長野,ANJ（P'sCUBE),マルハンメガシティ長野 データ一覧
2,アムズツインパーク,ANJ（P'sCUBE),アムズツインパーク データ一覧
3,キコーナ松戸店,プロテラス （テラスモバイル）,キコーナ松戸店 データ一覧
4,キコーナ新宿三丁目店,プロテラス （テラスモバイル）,キコーナ新宿三丁目店 データ一覧
...,...,...,...
0,エアポート777土岐店,グローリーナスカ（PAPIMO-NET),エアポート777土岐店 データ一覧
0,747三和店,ANJ（P'sCUBE),747三和店 データ一覧
0,RITZ高槻店,ANJ（P'sCUBE),RITZ高槻店 データ一覧
0,RITZ防府店,ANJ（P'sCUBE),RITZ防府店 データ一覧


In [54]:
concat_serch_df.to_csv('csv/convert_dmm_to_anaslo_completed_2023-06-21.csv')

In [49]:
serched_df_1

,0,1,2
0,マルハンメガシティ三島駿東,ANJ（P'sCUBE),マルハンメガシティ三島駿東 データ一覧
1,マルハンメガシティ長野,ANJ（P'sCUBE),マルハンメガシティ長野 データ一覧
2,アムズツインパーク,ANJ（P'sCUBE),アムズツインパーク データ一覧
3,キコーナ松戸店,プロテラス （テラスモバイル）,キコーナ松戸店 データ一覧
4,キコーナ新宿三丁目店,プロテラス （テラスモバイル）,キコーナ新宿三丁目店 データ一覧
...,...,...,...
917,ウイング金場店,プロテラス （テラスモバイル）,ウイング金場店 データ一覧
918,ウイング玉城店,プロテラス （テラスモバイル）,ウイング玉城店 データ一覧
919,ウイング四日市中央店,プロテラス （テラスモバイル）,ウイング四日市中央店 データ一覧
920,ウイング岩出店,プロテラス （テラスモバイル）,ウイング岩出店 データ一覧


In [50]:
#serched_df_1 = serched_df.reset_index(drop=True)
serched_df_1 = serched_df_1.rename(columns={0: 'DMM店舗名(変換前)',1:'取得ドメイン',2: 'アナスロ(変換後)'})
serched_df_1

,DMM店舗名(変換前),取得ドメイン,アナスロ(変換後)
0,マルハンメガシティ三島駿東,ANJ（P'sCUBE),マルハンメガシティ三島駿東 データ一覧
1,マルハンメガシティ長野,ANJ（P'sCUBE),マルハンメガシティ長野 データ一覧
2,アムズツインパーク,ANJ（P'sCUBE),アムズツインパーク データ一覧
3,キコーナ松戸店,プロテラス （テラスモバイル）,キコーナ松戸店 データ一覧
4,キコーナ新宿三丁目店,プロテラス （テラスモバイル）,キコーナ新宿三丁目店 データ一覧
...,...,...,...
917,ウイング金場店,プロテラス （テラスモバイル）,ウイング金場店 データ一覧
918,ウイング玉城店,プロテラス （テラスモバイル）,ウイング玉城店 データ一覧
919,ウイング四日市中央店,プロテラス （テラスモバイル）,ウイング四日市中央店 データ一覧
920,ウイング岩出店,プロテラス （テラスモバイル）,ウイング岩出店 データ一覧


In [43]:
serched_df_2.to_csv('csv/convert_dmm_to_anaslo_0923~2000.csv')

In [38]:
concat_serch_df[~concat_serch_df['アナスロ(変換後)'].str.contains('データ一覧')]

,DMM店舗名(変換前),取得ドメイン,アナスロ(変換後)
49,キコーナ香里園店,ANJ（P'sCUBE),大阪府
56,ガイア瑞江店,ANJ（P'sCUBE),アナスロ - 全国パチスロデータ・出玉情報
57,ガイアネクスト三ツ境Ⅰ,ANJ（P'sCUBE),神奈川県
58,ガイア西船橋店,ANJ（P'sCUBE),アナスロ - 全国パチスロデータ・出玉情報
59,メガガイア青森中央,ANJ（P'sCUBE),青森県
...,...,...,...
873,エンターテイメントオメガ堅田店,グローリーナスカ（PAPIMO-NET),2023/06/18 滋賀県 全台系と思われる機種があったお店 データ ...
877,yout勝川店,GOLUCK（台データ）,愛知県
885,Aパンハイ-ブリっヂ,GOLUCK（台データ）,2022/11/09 APANCLUBハイ-ブリっヂ データまとめ
899,丸三東萩店,ANJ（P'sCUBE),アナスロ - 全国パチスロデータ・出玉情報


In [22]:
convert_tenponame_dict_df = pd.DataFrame(dmm_and_anaslo_tenpo_name_dict.values(), index=dmm_and_anaslo_tenpo_name_dict.keys())
convert_tenponame_dict_df = convert_tenponame_dict_df.reset_index()
convert_tenponame_dict_df  = convert_tenponame_dict_df.rename(columns={'index': 'DMM(変換前)',0: 'アナスロ(変換後)'})
convert_tenponame_dict_df

,DMM(変換前),アナスロ(変換後)
0,マルハンメガシティ三島駿東,マルハンメガシティ三島駿東 データ一覧
1,マルハンメガシティ長野,マルハンメガシティ長野 データ一覧
2,アムズツインパーク,アムズツインパーク データ一覧
3,キコーナ松戸店,キコーナ松戸店 データ一覧
4,キコーナ新宿三丁目店,キコーナ新宿三丁目店 データ一覧
5,キコーナ府中店,キコーナ府中店 データ一覧
6,キコーナひたちなか店,キコーナひたちなか店 データ一覧
7,キコーナ水戸駅南店,キコーナ水戸駅南店 データ一覧
8,アプリイ菊川,アプリイ菊川 データ一覧
9,アプリイ富士川,アプリイ富士川 データ一覧


In [37]:
convert_tenponame_dict_df[~convert_tenponame_dict_df['アナスロ(変換後)'].str.contains('データ一覧')]

,DMM(変換前),アナスロ(変換後)


In [12]:
convert_tenponame_dict_df = convert_tenponame_dict_df[convert_tenponame_dict_df['アナスロ(変換後)'].str.contains('データ一覧')]
convert_tenponame_dict_df['アナスロ(変換後)'] = convert_tenponame_dict_df['アナスロ(変換後)'].apply(lambda x: x.replace(' データ一覧',''))
convert_tenponame_dict_df.to_csv(r'csv\dict_0602_昇天チャレンジ_アナスロ店舗名変換データ一覧.csv')
convert_tenponame_dict_df
#データ一覧のみを含む店舗名法人名とアナスロ店舗名の辞書型
extraxt_convert_tenponame_dict = dict(zip(convert_tenponame_dict_df['DMM(変換前)'], convert_tenponame_dict_df['アナスロ(変換後)']))
extraxt_convert_tenponame_dict

{'100℃大曽根店': '100℃大曽根店',
 '100℃守山店': '100℃守山店',
 '100万ドルアムアム店': 'よろこびの街100万ドルアムアム店',
 '100万ドル上田インター店': 'よろこびの街100万ドル上田インター店',
 '100万ドル中野店': 'よろこびの街100万ドル中野店',
 '123+Nはりまや橋店': '123+Nはりまや橋店',
 '123+N和歌山本店': '123+N和歌山本店',
 '123+N岡山本店': '123+N岡山本店',
 '123+N布施店': '123+N布施店',
 '123+N羽曳野店': '123+N羽曳野店',
 '123+Ｎ松原店': '123+N松原店'}

In [13]:
prefecture_name_list = '''北海道
青森県
岩手県
宮城県
秋田県
山形県
福島県
茨城県
栃木県
群馬県
埼玉県
千葉県
東京都
神奈川県
新潟県
富山県
石川県
福井県
山梨県
長野県
岐阜県
静岡県
愛知県
三重県
滋賀県
京都府
大阪府
兵庫県
奈良県
和歌山県
鳥取県
島根県
岡山県
広島県
山口県
徳島県
香川県
愛媛県
高知県
福岡県
佐賀県
長崎県
熊本県
大分県
宮崎県
鹿児島県
沖縄県'''.split('\n')

In [14]:
import requests
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import datetime
import time
import unicodedata
import string
import urllib
def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text
def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload)
def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files)
def TablePlot(df,outputPath,w,h,fontsize,title):
    plt.rcParams["font.size"] = fontsize
    plt.rcParams["font.weight"] = "bold"
    fig, ax = plt.subplots(figsize=(w,h))
    ax.axis('off')
    ax.table(cellText=df.values,
             colLabels=df.columns,
             loc='center',
             bbox=[0,0,1,1])
    ax.set_title(f'{title}',
             fontweight ="bold")
    plt.savefig(outputPath)


#print(tenpo_ichiran_df['ホール名'])
cols = ['機種名', '台番号', 'G数', '差枚', 'BB', 'RB', 'ART', 'BB確率', 'RB確率', 'ART確率','合成確率','店舗名']
ichiran_all_tennpo_df = pd.DataFrame(index=[], columns=cols)
yesterday = datetime.date.today() + datetime.timedelta(days=-1)
browser = webdriver.Chrome(ChromeDriverManager().install())
parlar_name_parlar_url_dict = {}
for prefecture_name in prefecture_name_list:
    url = f'https://ana-slo.com/%E3%83%9B%E3%83%BC%E3%83%AB%E3%83%87%E3%83%BC%E3%82%BF/{prefecture_name}/'
    browser.get(url)
    html = browser.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'lxml')
    for element in soup.find(class_="dataTable no-footer").find_all("a"):
        #print(element.string,element['href'])
        parlar_name = element.string
        parlar_name_url = element['href']
        parlar_name_parlar_url_dict[parlar_name] = parlar_name_url

        



====== WebDriver manager ======
Current google-chrome version is 114.0.5735
Get LATEST chromedriver version for 114.0.5735 google-chrome
Driver [C:\Users\slotd\.wdm\drivers\chromedriver\win32\114.0.5735.90\chromedriver.exe] found in cache
C:\Users\slotd\AppData\Local\Temp/ipykernel_43180/297629090.py:50: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


In [15]:
import csv
dct = parlar_name_parlar_url_dict
with open(r'csv/tenpo_name_tenpo_url_dict.csv', 'w') as f:  
    writer = csv.writer(f)
    for k, v in dct.items():
       writer.writerow([k, v])

In [16]:


tenpo_name_tenpo_url_df =pd.read_csv(r'csv\tenpo_name_tenpo_url_dict.csv', header=None, names=['店舗名','URL'])
tenpo_name_tenpo_url_df['url_店舗名'] = tenpo_name_tenpo_url_df['URL'].map(lambda x: urllib.parse.unquote((x.split('/')[-2])).replace('-データ一覧',''))
tenpo_name_tenpo_url_df.to_csv(r'csv/アナスロ店舗名デコードテーブル.csv',index=False)

In [80]:
tenpo_name_tenpo_url_df.sort_values('店舗名')

,店舗名,URL,url_店舗名
1779,100℃大曽根店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,100℃大曽根店
1780,100℃守山店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,100℃守山店
2813,123+Nはりまや橋店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,123nはりまや橋店
2514,123+N和歌山本店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,123n和歌山本店
2181,123+N大阪本店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,123n大阪本店
...,...,...,...
1219,阿佐ヶ谷オーシャン,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,阿佐ヶ谷オーシャン
1220,飯田橋プレサス,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,飯田橋プレサス
1428,鶴見UNO,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,鶴見uno
570,麗都宇都宮テクノ店,https://ana-slo.com/%e3%83%9b%e3%83%bc%e3%83%a...,麗都宇都宮テクノ店


In [ ]:

#マルハンを削除
tenpo_name_tenpo_url_df = tenpo_name_tenpo_url_df[~tenpo_name_tenpo_url_df['店舗名'].str.contains('マルハン')]
display(tenpo_name_tenpo_url_df)
anaslo_all_tenpo_name_list:list[str] = tenpo_name_tenpo_url_df['店舗名'].tolist()
anaslo_all_tenpo_name_list

In [ ]:
anaslo_maruhan_tenpo_name_list:list[str] = list(tenpo_name_tenpo_url_df[tenpo_name_tenpo_url_df['店舗名'].str.contains('マルハン')]['店舗名'])
print(len(anaslo_maruhan_tenpo_name_list))
anaslo_maruhan_tenpo_name_list

In [77]:
surplus_tenpo_name_list:list[str] = list(set(anaslo_all_tenpo_name_list) - set(convert_dmm_to_anaslo_tenponame_list))
print(len(surplus_tenpo_name_list))
print(sorted(surplus_tenpo_name_list))

636
['123+N大阪本店', '123+N東雲店', '123+N橿原店', '123CiTY!WAKAYAMA店', '123ファースト八尾店', '123三木店', '123五社店', '123千林店', '123和歌山インター店', '123塩屋店', '123天理店', '123宝塚店', '123岩岡店', '123川西店', '123弁天町店', '123摂津店', '123新三田店', '123横浜西口店', '123江坂店', '123泉佐野店', '123泉北店', '123田辺店', '123神戸店', '123緑橋店', '123論田店', '123長吉店', '123長居店', '123難波店', '21SEIKI北上', '7&8', 'A TIME 新開地店', 'A TIME 本店', 'A TIME 須磨パティオ店', 'ALEA WORLD舞屋店', 'APAN21勝川店', 'APANCLUBハイ-ブリっヂ', 'APULO811', 'APULO塩尻北インター店', 'B-MAX玖珠店', 'BBステーション日暮里店', 'BEAM YAME', 'BEAM串間', 'BOOM天神本店', 'CLUB-D', 'CPセントラルパーク大分本店', 'Cochi・Pachi', 'DAMZ inn 坂町店', 'DAMZ inn 紫竹店', 'DAMZ上越店', 'DAMZ両津店', 'DAMZ六日町店', 'DAMZ十日町きらめき店', 'DAMZ塩沢店', 'DAMZ寺尾店', 'DAMZ小出店', 'DAMZ小千谷店', 'DAMZ木戸店', 'DAMZ村上店', 'DAMZ柿崎店', 'DAMZ見附今町店', 'DELFEEL NEW TOKYO', 'DOME大津', 'DON+', 'Dee1', 'Dステーション八千代店', 'Dステーション大村店', 'EVO直方', 'FAST天白店', 'FAST東浦店', 'FORUM-1神田店', 'G&L7', 'G&L一宮店', 'G7', 'GLOBE', 'GM2', 'GOGO千代田', 'GRAND SEPIA', 'HAP-1-NHWARABI', 'JAM登戸', 'JP-7館林店', 'JUMBO上大岡', 'KB SUPER STADIUM勝浦店',

In [106]:
l1_l2_and = set(anaslo_all_tenpo_name_list) & set(convert_dmm_to_anaslo_tenponame_list)
print(len(l1_l2_and))
# {'c', 'b'}

2247


In [ ]:
for name in sorted(surplus_tenpo_name_list):
    print(name)

In [82]:
print(len(convert_dmm_to_anaslo_tenponame_list))

2352


In [ ]:
#https://hall-navi.com/hole_view?hid=239083100000015111

In [ ]:
#マルハンを抜く
concat_serch_df_1 = concat_serch_df[~concat_serch_df['アナスロ(変換後)'].str.contains('マルハン')]
concat_serch_df_1 = concat_serch_df_1[concat_serch_df_1['アナスロ(変換後)'].str.contains('データ一覧')]
concat_serch_df_1['アナスロ(変換後)'] = concat_serch_df_1['アナスロ(変換後)'].map(lambda x: x.replace(' データ一覧', ''))
convert_dmm_to_anaslo_tenponame_list:list[str] = concat_serch_df_1['アナスロ(変換後)'].tolist()
print(len(convert_dmm_to_anaslo_tenponame_list))
sorted(convert_dmm_to_anaslo_tenponame_list)

In [88]:
anaslo_scraping_error_tenpo_name_dict = {}
browser = webdriver.Chrome(ChromeDriverManager().install())
non_extract_serched_df = pd.DataFrame(index=[], columns=[])
dmm_and_anaslo_tenpo_name_dict = {}
#no_scraping_anaslo_tenpo_name = []
i = 0
for tenpo_name in sorted(surplus_tenpo_name_list):
    try:
        url = f"https://www.google.com/search?q=アナスロ  {tenpo_name} データ一覧&hl=ja&source=hp/"
        #print('データ取得中...30秒ほど時間がかかります。')
        browser.get(url)
        html = browser.page_source.encode('utf-8')
        elems_rec = browser.find_elements_by_class_name('MjjYud')
        elems_rec[0].text.split('\n')
        #print('アナスロと出る部分',elems_rec[0].text.split('\n')[1])

        scraping_anaslo_name = elems_rec[0].text.split('\n')[0]
        dmm_and_anaslo_tenpo_name_dict[dmm_tenpo_name] = scraping_anaslo_name
        non_extract_serched_df = pd.concat([non_extract_serched_df,pd.DataFrame([[tenpo_name,scraping_anaslo_name]])],axis=0)
        time.sleep(2)
        i += 1
        # if i > 3:
        #     break
    except Exception as e:
        anaslo_scraping_error_tenpo_name_dict[dmm_tenpo_name] =  e
        print('エラー店舗',dmm_tenpo_name,e)
        continue


    



====== WebDriver manager ======
Current google-chrome version is 114.0.5735
Get LATEST chromedriver version for 114.0.5735 google-chrome
Driver [C:\Users\slotd\.wdm\drivers\chromedriver\win32\114.0.5735.90\chromedriver.exe] found in cache
C:\Users\slotd\AppData\Local\Temp/ipykernel_43180/132945816.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\slotd\AppData\Local\Temp/ipykernel_43180/132945816.py:13: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  elems_rec = browser.find_elements_by_class_name('MjjYud')


In [94]:
non_extract_serched_df.rename(columns={0: "DMM未取得アナスロ店舗名",1: "検索結果アナスロ店舗名"}, inplace=True)


In [95]:
non_extract_serched_df

,DMM未取得アナスロ店舗名,検索結果アナスロ店舗名
0,123+N大阪本店,123+N大阪本店 データ一覧
0,123+N東雲店,123+N東雲店 データ一覧
0,123+N橿原店,123+N橿原店 データ一覧
0,123CiTY!WAKAYAMA店,123CiTY!WAKAYAMA店 データ一覧
0,123ファースト八尾店,123ファースト八尾店 データ一覧
...,...,...
0,神志山KBシティ,神志山KBシティ データ一覧
0,稲口会館,稲口会館 データ一覧
0,蕨トップス,蕨トップス データ一覧
0,遊スポットボンボン,遊スポットボンボン データ一覧


In [105]:
non_extract_serched_df.to_csv('csv/アナスロにあってDMMアプリにない店舗名一覧.csv')

In [19]:
convert_tenponame_dict_df = pd.DataFrame(dmm_and_anaslo_tenpo_name_dict.values(), index=dmm_and_anaslo_tenpo_name_dict.keys())
convert_tenponame_dict_df = convert_tenponame_dict_df.reset_index()
convert_tenponame_dict_df  = convert_tenponame_dict_df.rename(columns={'index': '変換前',0: '変換後'})
#convert_tenponame_dict_df['変換後'] = convert_tenponame_dict_df['変換後'].apply(lambda x: x.replace(' データ一覧',''))
convert_tenponame_dict_df.to_csv(r'csv\dict_0503_アナスロ店舗名変換データ一覧.csv')
convert_tenponame_dict_df

,変換前,変換後
0,100℃大曽根店,100℃大曽根店 データ一覧
1,100℃守山店,100℃守山店 データ一覧
2,100万ドルアムアム店,よろこびの街100万ドルアムアム店 データ一覧
3,100万ドル上田インター店,よろこびの街100万ドル上田インター店 データ一覧
4,100万ドル中野店,よろこびの街100万ドル中野店 データ一覧
5,123+Nはりまや橋店,123+Nはりまや橋店 データ一覧
6,123+N和歌山本店,123+N和歌山本店 データ一覧
7,123+N岡山本店,123+N岡山本店 データ一覧
8,123+N布施店,123+N布施店 データ一覧
9,123+N羽曳野店,123+N羽曳野店 データ一覧


In [ ]:
#明日の生成

import datetime
target_day = datetime.date.today() + datetime.timedelta(days=1)
print(target_day)

#明日の日付を文字列に変換

print(target_day.strftime('%Y/%m/%d'))

In [112]:
len(target_day_df['店舗名'].unique())

1978

In [127]:
l1_l2_and = set(surplus_tenpo_name_list) & set(latest_tenpo_name_list)
print(len(l1_l2_and))

440


In [ ]:
for x in sorted(l1_l2_and):
    print(x)

In [ ]:
surplus_tenpo_name_list

In [ ]:
latest_tenpo_name_list = list(set(tenpo_name_list))
latest_tenpo_name_list

In [129]:
len(list(set(tenpo_name_list)))

2621

In [123]:
tenpo_name_list = []
for i in range(1, 12):
    target_day = datetime.date.today() - datetime.timedelta(days=i)
    print(target_day)
    target_day_str = target_day.strftime('%Y-%m-%d')
    target_day_df = pd.read_csv(f'csv/{target_day_str}_anaslo.csv')
    display(target_day_df)
    for tenpo_name in list(target_day_df['店舗名'].unique()):
        tenpo_name_list.append(tenpo_name)
    #break


2023-06-22


,UUID,日付,Nの日,都道府県,店舗名,機種名,台番号,末尾,G数,差枚,BB,RB,BB確率,RB確率,合成確率,x_value,y_value
0,4e26789c-50b1-4dfe-a5b1-a076c35e481f,2023/06/22 00:00:00,2,北海道,KEIZ手稲店,アイムジャグラーEX-TP,909,9,4680.0,-260,17.0,12.0,1/275.3,1/390.0,1/161.4,"[0.5, 0.5, 37.5, 47.5, 58.5, 78.5, 89.5, 99.5,...","[0, 0, -29, -29, -76, -76, -76, -17, 192, 41, ..."
1,00342e33-3c85-485e-b424-a0a6f80becf0,2023/06/22 00:00:00,2,北海道,KEIZ手稲店,アイムジャグラーEX-TP,910,0,1706.0,-759,5.0,2.0,1/341.2,1/853.0,1/243.7,"[0.5, 0.5, 37.5, 58.5, 68.5, 78.5, 89.5, 99.5,...","[0, 0, -58, -58, -105, -360, -384, -430, -430,..."
2,5e44ba77-67a7-4fef-9057-ede13ae236e4,2023/06/22 00:00:00,2,北海道,KEIZ手稲店,アイムジャグラーEX-TP,911,1,2669.0,415,13.0,3.0,1/205.3,1/889.7,1/166.8,"[0.5, 0.5, 37.5, 47.5, 58.5, 68.5, 99.5, 109.5...","[0, 0, 0, -12, -81, -105, -105, 174, 779, 1128..."
3,877532ec-822f-452e-a837-390ec27d3943,2023/06/22 00:00:00,2,北海道,KEIZ手稲店,アイムジャグラーEX-TP,912,2,914.0,-426,3.0,0.0,1/304.7,1/0.0,1/304.7,"[0.5, 0.5, 37.5, 78.5, 89.5, 99.5, 109.5, 182....","[0, 0, -122, -122, -29, -29, -157, -157, -285,..."
4,5f9d4fb1-aac8-4093-8ede-cafac84b9c7e,2023/06/22 00:00:00,2,北海道,KEIZ手稲店,アイムジャグラーEX-TP,913,3,6191.0,-340,21.0,22.0,1/294.8,1/281.4,1/144.0,"[0.5, 0.5, 68.5, 78.5, 89.5, 99.5, 109.5, 120....","[0, 0, 0, -99, 76, 285, 308, 378, 169, 378, 54..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445492,b590ebf1-e2ce-49c0-99dd-e67d34a99015,2023/06/22 00:00:00,2,沖縄県,名宝具志川店,新ハナビ,588,8,760.0,446,5.0,2.0,1/152.0,1/380.0,1/108.6,"[1687392000, 1687394534, 1687431584, 168744250...","[0, 0, 109.99999993831887, 73.99999998391999, ..."
445493,eca7a53a-4349-4747-b6df-18454328e481,2023/06/22 00:00:00,2,沖縄県,名宝具志川店,ひぐらしのなく頃に祭2,600,0,417.0,-348,0.0,1.0,1/0.0,1/417.0,1/417.0,"[1687392000, 1687394534, 1687407794, 1687409627]","[0, 0, -142.9999998965178, -347.99999990863955]"
445494,08f8ba29-ead2-4d18-a23d-6563873ee17f,2023/06/22 00:00:00,2,沖縄県,名宝具志川店,麻雀格闘倶楽部 真,601,1,1894.0,-151,9.0,3.0,1/210.4,1/631.3,1/157.8,"[1687392000, 1687394534, 1687409627, 168741143...","[0, 0, 2.9999999961999047, -365.99999988583886..."
445495,338f5b22-6b62-4eec-b09b-c88c6bd9ad3e,2023/06/22 00:00:00,2,沖縄県,名宝具志川店,とある科学の超電磁砲,718,8,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0,"[1687392000, 1687394534]","[0, 0]"


2023-06-21


,UUID,日付,Nの日,都道府県,店舗名,機種名,台番号,末尾,G数,差枚,BB,RB,BB確率,RB確率,合成確率,x_value,y_value
0,507d4895-ecc3-445a-ba7a-dccfe247ac71,2023/06/21 00:00:00,1,北海道,BIG大王東店,マイジャグラーV,726,6,8945.0,1301,38.0,32.0,1/235.4,1/279.5,1/127.8,"[0, 3, 6, 9, 12, 15, 17, 20, 26, 29, 35, 38, 4...","[0, 0, 0, 0, 0, 0, 0, -41, -41, 123, -178, 479..."
1,6849baf0-2818-444f-a9c3-5a102351d13a,2023/06/21 00:00:00,1,北海道,BIG大王東店,マイジャグラーV,727,7,7216.0,-1089,19.0,28.0,1/379.8,1/257.7,1/153.5,"[0, 3, 6, 9, 12, 15, 17, 20, 26, 29, 35, 38, 4...","[0, 0, 0, 0, 0, 0, 0, 104, -15, -134, -224, -3..."
2,64f1c385-622b-4dc8-8cd3-79ada42c5d6d,2023/06/21 00:00:00,1,北海道,BIG大王東店,マイジャグラーV,728,8,8565.0,1120,34.0,29.0,1/251.9,1/295.3,1/136.0,"[0, 3, 6, 9, 12, 15, 17, 20, 23, 26, 29, 35, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 433, 373, 75, 134,..."
3,ae5372df-7963-40b3-ba8f-52f1930556a6,2023/06/21 00:00:00,1,北海道,BIG大王東店,マイジャグラーV,730,0,1877.0,-283,7.0,5.0,1/268.1,1/375.4,1/156.4,"[0, 3, 6, 9, 12, 15, 17, 20, 26, 29, 38, 43, 5...","[0, 0, 0, 0, 0, 0, 0, -45, -75, -224, -224, -2..."
4,a4034645-e19a-4384-b5fe-86c403c89749,2023/06/21 00:00:00,1,北海道,BIG大王東店,マイジャグラーV,731,1,7733.0,1794,31.0,34.0,1/249.5,1/227.4,1/119.0,"[0, 3, 6, 9, 12, 15, 17, 20, 26, 29, 35, 38, 4...","[0, 0, 0, 0, 0, 0, 0, 178, 288, 425, 507, 1027..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382806,e29a418a-860b-437b-a85f-49d7baca699a,2023/06/21 00:00:00,1,沖縄県,名宝具志川店,新ハナビ,588,8,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0,"[1687305600, 1687308136]","[0, 0]"
382807,a2e8ab3c-f59a-4324-b4da-1294c3c6e04b,2023/06/21 00:00:00,1,沖縄県,名宝具志川店,ひぐらしのなく頃に祭2,600,0,377.0,41,1.0,1.0,1/377.0,1/377.0,1/188.5,"[1687305600, 1687308136, 1687312135, 1687314207]","[0, 0, 20.99999997339933, 40.99999990923752]"
382808,576be7c2-e5ff-4515-ba11-cdc815d43317,2023/06/21 00:00:00,1,沖縄県,名宝具志川店,麻雀格闘倶楽部 真,601,1,242.0,509,1.0,1.0,1/242.0,1/242.0,1/121.0,"[1687305600, 1687308136, 1687316001, 1687317846]","[0, 0, -125.99999995687949, 508.99999989884026]"
382809,1d831f39-71e8-42a7-82e1-32b88c3b3901,2023/06/21 00:00:00,1,沖縄県,名宝具志川店,とある科学の超電磁砲,718,8,0.0,0,0.0,0.0,1/0.0,1/0.0,1/0.0,"[1687305600, 1687308136]","[0, 0]"


2023-06-20


...

,UUID,日付,Nの日,都道府県,店舗名,機種名,台番号,末尾,G数,差枚,BB,RB,BB確率,RB確率,合成確率,x_value,y_value
0,b0f6e0c5-2cd1-46bb-93c4-3c92b388a790,2023/06/14 00:00:00,4,北海道,KEIZ手稲店,アイムジャグラーEX-TP,909,9,6057.0,-420,19.0,27.0,1/318.8,1/224.3,1/131.7,"[0.5, 0.5, 37.5, 47.5, 58.5, 68.5, 78.5, 89.5,...","[0, 0, 35, -267, -477, -500, -500, -593, -593,..."
1,b67b37c8-f787-45cb-b56c-4bee0d491244,2023/06/14 00:00:00,4,北海道,KEIZ手稲店,アイムジャグラーEX-TP,910,0,2968.0,1164,15.0,6.0,1/197.9,1/494.7,1/141.3,"[0.5, 0.5, 37.5, 47.5, 89.5, 99.5, 109.5, 120....","[0, 0, 0, -12, -12, 221, 174, 942, 1105, 1453,..."
2,4206ea19-14c4-418f-8a8f-a6fa871c7e6b,2023/06/14 00:00:00,4,北海道,KEIZ手稲店,アイムジャグラーEX-TP,911,1,2437.0,-470,7.0,8.0,1/348.1,1/304.6,1/162.5,"[0.5, 0.5, 37.5, 47.5, 58.5, 68.5, 78.5, 109.5...","[0, 0, 0, -17, -17, -41, -64, -64, -169, -169,..."
3,06228571-562f-4928-835d-177de1496a19,2023/06/14 00:00:00,4,北海道,KEIZ手稲店,アイムジャグラーEX-TP,912,2,2413.0,-964,5.0,4.0,1/482.6,1/603.2,1/268.1,"[0.5, 0.5, 37.5, 58.5, 68.5, 78.5, 89.5, 99.5,...","[0, 0, -6, -6, -76, -76, -110, -110, -250, -25..."
4,9d57e560-e297-4b0f-b934-8997d7b46a1d,2023/06/14 00:00:00,4,北海道,KEIZ手稲店,アイムジャグラーEX-TP,913,3,4403.0,-942,14.0,9.0,1/314.5,1/489.2,1/191.4,"[0.5, 0.5, 78.5, 89.5, 99.5, 109.5, 120.5, 130...","[0, 0, 0, -29, -29, -157, 76, 87, 17, 87, -76,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436352,795ec891-cab9-46a6-ae39-5090f1ccfbcd,2023/06/14 00:00:00,4,沖縄県,名宝具志川店,アラジンAクラシック,586,6,1538.0,-187,19.0,2.0,1/80.9,1/769.0,1/73.2,"[1686700800, 1686703336, 1686734726, 168673655...","[0, 0, -160.99999999020073, 135.99999998304037..."
436353,88485891-497c-4e2c-bd87-1eab4db53b52,2023/06/14 00:00:00,4,沖縄県,名宝具志川店,バーサスリヴァイズ,587,7,1546.0,-533,3.0,3.0,1/515.3,1/515.3,1/257.7,"[1686700800, 1686703336, 1686731085, 168673292...","[0, 0, -32.99999995819894, -334.9999998862788,..."
436354,d5d44d24-2e17-43b2-a665-8427e5f349eb,2023/06/14 00:00:00,4,沖縄県,名宝具志川店,新ハナビ,588,8,572.0,-418,1.0,0.0,1/572.0,1/0.0,1/572.0,"[1686700800, 1686703336, 1686729311, 168673108...","[0, 0, 47.999999939198474, -164.99999990747827..."
436355,1fe6e185-c8f8-4884-aa61-43953daf5129,2023/06/14 00:00:00,4,沖縄県,名宝具志川店,ひぐらしのなく頃に祭2,600,0,9469.0,2934,30.0,22.0,1/315.6,1/430.4,1/182.1,"[1686700800, 1686703336, 1686705232, 168670727...","[0, 0, -47.999999939198474, -270.9999999285195..."


2023-06-13


,UUID,日付,Nの日,都道府県,店舗名,機種名,台番号,末尾,G数,差枚,BB,RB,BB確率,RB確率,合成確率,x_value,y_value
0,22ed84bb-b2d9-4330-b8ad-ce417ae867d1,2023/06/13 00:00:00,3,北海道,MEGA M'S1121西岡店,スマスロ北斗の拳,581,1,1959.0,-678,17.0,4.0,1/115.2,1/489.8,1/93.3,"[9, 9.1, 9.3, 9.5, 9.6, 9.8, 10, 10.1, 10.3, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,23b1b91e-7120-428d-a172-24ac8a403f23,2023/06/13 00:00:00,3,北海道,MEGA M'S1121西岡店,スマスロ北斗の拳,582,2,1826.0,-442,16.0,6.0,1/114.1,1/304.3,1/83.0,"[9, 9.1, 9.3, 9.5, 9.6, 9.8, 10, 10.1, 10.3, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,bdcfcb51-f168-4494-96fd-44c0d402b3f1,2023/06/13 00:00:00,3,北海道,MEGA M'S1121西岡店,スマスロ北斗の拳,583,3,2269.0,-843,19.0,3.0,1/119.4,1/756.3,1/103.1,"[9, 9.1, 9.3, 9.5, 9.6, 9.8, 10, 10.1, 10.3, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,f4fc92eb-ab91-4f52-b1a6-a7cfa10489fa,2023/06/13 00:00:00,3,北海道,MEGA M'S1121西岡店,スマスロ北斗の拳,584,4,480.0,-289,2.0,2.0,1/240.0,1/240.0,1/120.0,"[9, 9.1, 9.3, 9.5, 9.6, 9.8, 10, 10.1, 10.3, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,d0ddff7a-3f0f-489e-ba9a-8eec64b94438,2023/06/13 00:00:00,3,北海道,MEGA M'S1121西岡店,スマスロ北斗の拳,585,5,1353.0,4089,41.0,5.0,1/33.0,1/270.6,1/29.4,"[9, 9.1, 9.3, 9.5, 9.6, 9.8, 10, 10.1, 10.3, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434300,fcd412c6-1a72-4d07-8d99-0275635c6f7d,2023/06/13 00:00:00,3,沖縄県,名宝具志川店,アラジンAクラシック,586,6,33.0,-48,0.0,0.0,1/0.0,1/0.0,1/0.0,"[1686614400, 1686616934, 1686654034, 1686655851]","[0, 0, -16.99999993963829, -47.999999939198474]"
434301,d21473d3-eebd-4cae-b659-f67521bc0250,2023/06/13 00:00:00,3,沖縄県,名宝具志川店,バーサスリヴァイズ,587,7,727.0,402,4.0,1.0,1/181.8,1/727.0,1/145.4,"[1686614400, 1686616934, 1686644829, 168664670...","[0, 0, -9.999999909677337, 551.9999999996836, ..."
434302,45fdb206-5eda-4af2-978c-755a11b57957,2023/06/13 00:00:00,3,沖縄県,名宝具志川店,新ハナビ,588,8,371.0,-152,1.0,2.0,1/371.0,1/185.5,1/123.7,"[1686614400, 1686616934, 1686657657, 168666127...","[0, 0, -227.99999994415978, -91.99999996111943..."
434303,7604ec4c-dfc2-4d4f-bc68-7a860dd818b3,2023/06/13 00:00:00,3,沖縄県,名宝具志川店,ひぐらしのなく頃に祭2,600,0,2011.0,-1051,5.0,2.0,1/402.2,1/1005.5,1/287.3,"[1686614400, 1686616934, 1686618855, 168662093...","[0, -145.9999998927177, -513.9999999701624, -4..."


2023-06-12


,UUID,日付,Nの日,都道府県,店舗名,機種名,台番号,末尾,G数,差枚,BB,RB,BB確率,RB確率,合成確率,x_value,y_value
0,21c82a84-6f40-43b9-8f94-d84d01d2f07e,2023/06/12 00:00:00,2,北海道,BIG大王東店,マイジャグラーV,726,6,4904.0,333,20.0,13.0,1/245.2,1/377.2,1/148.6,"[0, 3, 7, 10, 13, 16, 19, 22, 38, 41, 48, 51, ...","[0, 0, 0, 0, 0, 0, 0, 13, 200, 93, 93, -253, -..."
1,9f32f219-7f07-4150-ba36-92c62340fe65,2023/06/12 00:00:00,2,北海道,BIG大王東店,マイジャグラーV,727,7,2384.0,-27,9.0,6.0,1/264.9,1/397.3,1/158.9,"[0, 3, 7, 10, 13, 16, 19, 57, 61, 67, 70, 95, ...","[0, 0, 0, 0, 0, 0, -13, 173, 253, -67, -93, -1..."
2,84c12b6f-fbc1-438e-8f8a-be02d5177ebd,2023/06/12 00:00:00,2,北海道,BIG大王東店,マイジャグラーV,728,8,1567.0,-600,3.0,5.0,1/522.3,1/313.4,1/195.9,"[0, 3, 7, 10, 13, 16, 19, 22, 26, 29, 32, 35, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,ef027283-e7f0-4f95-8012-f95e72bf35a3,2023/06/12 00:00:00,2,北海道,BIG大王東店,マイジャグラーV,730,0,3311.0,-520,10.0,9.0,1/331.1,1/367.9,1/174.3,"[0, 3, 7, 10, 13, 19, 22, 29, 32, 38, 89, 95, ...","[0, 0, 0, 93, 147, -67, -200, 0, -200, -360, -..."
4,25c19338-69f0-4429-a507-c8740764a05e,2023/06/12 00:00:00,2,北海道,BIG大王東店,マイジャグラーV,731,1,1322.0,-600,4.0,1.0,1/330.5,1/1322.0,1/264.4,"[0, 3, 7, 10, 13, 16, 19, 48, 51, 57, 61, 67, ...","[0, 0, 0, 0, 0, 0, 13, 0, 333, 280, -13, -40, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494035,1a854d86-14b0-4e73-a2f9-ffd1ba02fc07,2023/06/12 00:00:00,2,沖縄県,名宝具志川店,アラジンAクラシック,586,6,79.0,-144,0.0,0.0,1/0.0,1/0.0,1/0.0,"[1686528000, 1686530532, 1686534446, 168653652...","[0, 0, -28.9999999244379, -90.99999992355826, ..."
494036,a82491e2-e7fc-4037-9cfa-2199742a1249,2023/06/12 00:00:00,2,沖縄県,名宝具志川店,バーサスリヴァイズ,587,7,1.0,-2,0.0,0.0,1/0.0,1/0.0,1/0.0,"[1686528000, 1686530532, 1686558349]","[0, 0, -1.9999999586387642]"
494037,c0a29ad8-7354-4512-b899-24ff65ddaf11,2023/06/12 00:00:00,2,沖縄県,名宝具志川店,新ハナビ,588,8,210.0,62,1.0,0.0,1/210.0,1/0.0,1/210.0,"[1686528000, 1686530532, 1686562035, 168656394...","[0, 0, 96.99999991595809, 109.99999993831887, ..."
494038,0c06ad88-14c6-4d51-94af-f5c647b5582c,2023/06/12 00:00:00,2,沖縄県,名宝具志川店,ひぐらしのなく頃に祭2,600,0,238.0,101,1.0,0.0,1/238.0,1/0.0,1/238.0,"[1686528000, 1686530532, 1686569454, 1686571253]","[0, 0, 165.99999994503938, 100.99999994971913]"


In [11]:
tenpo_name_list ='''123+N和歌山本店
123+N大阪本店
123+N布施店
123+N羽曳野店
123横浜西口店
123長吉店
123門真店
123難波店
123鶴橋店
123+N橿原店
BEAM HARA
Dステーション仙台東店
Dステーション上田店
Dステーション新小岩店
EVO2
GRAND G-ONE甲賀水口店
HYPER ARROW美原店
KING BOSS 1000
MEGA BEAM ASAKURA 999
メガコンコルド1020刈谷知立店
メガコンコルド1111BLAZE店
メガコンコルド1280稲沢店
MGM小山本郷店
PIA京急川崎
PIA八王子
PIA川崎ダイス
PIA柏 スロット館
PIA横浜モアーズ
PIA町田新館
スーパーDステーション上里店
スーパーDステーション立川店
スーパーDステーション鴻巣店
WIN-WIN ONE
アスカ熊谷店
アスカ狭山店
アビバ湘南台店
アビバ関内
ジュラク王子店
エクスアリーナ柏
エクスアリーナ東京
オゼック国分寺北口
キコーナ吹田店
キコーナ守口店
キコーナ小田原扇町店
キコーナ海老名店
キコーナ茨木店
グランキコーナ堺店
グランキコーナ泉佐野店
グランキコーナ相模原店
グランドオータ瀬谷店
サンパレス環七大杉店
ZAP PRESTIGE
ジアスセンター南
ジャムフレンドクラブ朝霞店
スロット123梶取店
テンガイ・与次郎店
パラッツォ三郷中央店
ビッグアップル.加古川
ピーアーク三田
ピーアークピーくんステーション
ピーアーク相模原
ピーアーク相模大野
ピーアーク竹ノ塚スタジオ
ピーアーク草加
メガフェイス1180座間
プレイランド第一平和
マルハンいわき店
マルハン草薙アピア店
マルハン青梅新町店
ミリオン習志野店
メガコンコルド800春日井店+
レッドロック学芸大学駅前店
UNO三ノ輪
上尾UNO
UNO上福岡
UNO中山
ZoRoN中山
UNO仙川
出玉王田無店
UNO北越谷
UNO南柏
UNO新橋
第一プラザ西浦和店
UNO本八幡
楽園上新屋店
楽園川崎店
楽園沼津店
楽園浜松ザザシティ店
楽園袋井店
UNO清瀬
UNO秋葉原
UNO鶴見
BEAM by HIKARI'''.split('\n')
len(tenpo_name_list)

93

In [20]:
convert_tenponame_dict_df['変換後']

0                100℃大曽根店 データ一覧
1                 100℃守山店 データ一覧
2       よろこびの街100万ドルアムアム店 データ一覧
3     よろこびの街100万ドル上田インター店 データ一覧
4         よろこびの街100万ドル中野店 データ一覧
5             123+Nはりまや橋店 データ一覧
6              123+N和歌山本店 データ一覧
7               123+N岡山本店 データ一覧
8                123+N布施店 データ一覧
9               123+N羽曳野店 データ一覧
10               123+N松原店 データ一覧
Name: 変換後, dtype: object

In [ ]:
anaslo_scraping_error_tenpo_name_list = []
browser = webdriver.Chrome(ChromeDriverManager().install())
serched_df = pd.DataFrame(index=[], columns=[])
# for target_day in list(sammy_sf_report_df['イベント日'].unique()):
#     print(target_day)
error_tenpo_name_list = []
target_day_list = ['2023-06-02','2023-06-03']
anaslo_scraping_error_tenpo_name_list  = []
for tenpo_name in convert_tenponame_dict_df['変換後']:
    anaslo_parlar_name_url = parlar_name_parlar_url_dict[tenpo_name]
    anaslo_url_parlarname =urllib.parse.unquote((anaslo_parlar_name_url.split('/')[-2])).replace('-データ一覧','')
    for date in target_day_list:
        try:
            url = f"https://ana-slo.com/{date}-{anaslo_url_parlarname}-data/"
            print(url)
            #print('データ取得中...30秒ほど時間がかかります。')
            browser.get(url)
            html = browser.page_source.encode('utf-8')
            dfs = pd.read_html(html)
            #display(tenpo_df)
            #time.sleep(1)
            elements = browser.find_element(By.CLASS_NAME, 'st-catgroup')
            prefecture = elements.text.split(' ')[-1]
            for df in  dfs:
            #print(df.columns)
                if '機種名' in list(df.columns):
                    ichiran_df = df
                    ichiran_df['店舗名'] = tenpo_name
                    #ichiran_df['都道府県'] = prefecture
                    ichiran_df['機種名'] = ichiran_df['機種名'].map(removal_text)
                    first_column = ichiran_df.pop('店舗名')
                    ichiran_df.insert(0,'店舗名',first_column)
                    break
            tenpo_day_data_df = ichiran_df
            tenpo_day_data_df = tenpo_day_data_df.sort_values('台番号')
            tenpo_day_data_df['日付'] = f"{date}"
            serched_df = pd.concat([serched_df,tenpo_day_data_df],axis=0)
        except Exception as e:
            anaslo_scraping_error_tenpo_name_list.append(tenpo_name)
            print('エラー店舗',tenpo_name,e)
            continue


In [24]:
serched_df

,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
339,123+N和歌山本店,モンキーターンIV,705,102,-185.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-06-03
340,123+N和歌山本店,モンキーターンIV,706,965,-641.0,0,1,2.0,1/321.7,1/0.0,1/965.0,1/482.5,2023-06-03
341,123+N和歌山本店,モンキーターンIV,707,287,-126.0,0,1,2.0,1/95.7,1/0.0,1/287.0,1/143.5,2023-06-03
342,123+N和歌山本店,戦姫絶唱シンフォギア勇気の歌,708,1712,580.0,0,6,21.0,1/63.4,1/0.0,1/285.3,1/81.5,2023-06-03
343,123+N和歌山本店,戦姫絶唱シンフォギア勇気の歌,709,1310,-1534.0,0,4,2.0,1/218.3,1/0.0,1/327.5,1/655.0,2023-06-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,BEAM by HIKARI,沖ドキGOLD,1172,727,1458.0,10,4,NaN,1/51.9,1/72.7,1/181.8,NaN,2023-06-02
132,BEAM by HIKARI,沖ドキGOLD,1173,1877,4039.0,32,5,NaN,1/50.7,1/58.7,1/375.4,NaN,2023-06-02
133,BEAM by HIKARI,沖ドキGOLD,1175,408,-546.0,0,1,NaN,1/408.0,1/0.0,1/408.0,NaN,2023-06-02
134,BEAM by HIKARI,沖ドキGOLD,1176,1697,1169.0,17,3,NaN,1/84.8,1/99.8,1/565.7,NaN,2023-06-02


In [29]:
serched_df['日付'] = serched_df['日付'].astype(str)

In [33]:
target_day_list = ['2023-06-03','2023-06-02']

In [36]:
target_day_list
for date_str in target_day_list:
    serched_df[serched_df['日付'] == date_str].to_csv(fr'csv/{date_str}_scraping_anaslo.csv')

In [60]:
concat_data_df = pd.DataFrame(index=[], columns=[])

for date_str in target_day_list:
    csv_file_name  = f'csv/{date_str}_scraping_anaslo.csv'
    data_df =  pd.read_csv(csv_file_name)
    concat_data_df=  pd.concat([concat_data_df, data_df], axis=0)



In [49]:
concat_data_df['店舗名'].unique()

array(['123+N和歌山本店', '123+N大阪本店', '123+N布施店', '123+N羽曳野店', '123+N松原店',
       '123岩岡店', '123横浜西口店', '123長吉店', '123門真店', '123難波店', '123鶴橋店',
       'BEAM HARA', 'GRAND G-ONE甲賀水口店', 'HYPER ARROW美原店',
       'KING BOSS 1000', 'メガコンコルド1020刈谷知立店', 'メガコンコルド1111BLAZE店',
       'MGM小山本郷店', 'PIA京急川崎', 'PIA八王子', 'PIA川崎ダイス', 'PIA柏 スロット館',
       'PIA横浜モアーズ', 'PIA町田新館', 'スーパーDステーション入間店', 'スーパーDステーション鴻巣店',
       'WIN-WIN ONE', 'アスカ狭山店', 'アビバ湘南台店', 'アビバ関内', 'ジュラク王子店',
       'アムズガーデン石巻湊店', 'ウエスタン環七南葛西店', 'エクスアリーナ柏', 'エクスアリーナ東京', 'キコーナ吹田店',
       'キコーナ守口店', 'キコーナ小田原扇町店', 'キコーナ海老名店', 'グランキコーナ堺店', 'グランキコーナ相模原店',
       'グランドオータ瀬谷店', 'スロットサントロペ横須賀中央', 'サンパレス環七大杉店', 'ジャムフレンドクラブ朝霞店',
       'スロット123梶取店', 'スーパーライブガーデン行田店', 'テンガイ・与次郎店', 'ニューヨークニューヨーク泉店',
       'ニューヨークニューヨーク臨海店', 'パラッツォ三郷中央店', 'パラッツォ鳩ヶ谷店', 'パールショップともえセンター南店',
       'パールショップともえ市原店', 'パールショップともえ松戸店', 'ビッグアップル.加古川', 'ビッグアップル．秋葉原店',
       'ピーアーク三田', 'ピーアーク千葉駅前', 'ピーアーク相模原', 'ピーアーク相模大野', 'ピーアーク竹ノ塚スタジオ',
       'ピーアーク草加', 'プレイランド第一平和', 'マルハンな

In [39]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
#認証情報設定
#ダウンロードしたjsonファイル名をクレデンシャル変数に設定（秘密鍵、Pythonファイルから読み込みしやすい位置に置く）
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'sercret_json\twitteranalytics-jsonsercretkey.json', scope)
gc = gspread.authorize(credentials)
workbook = gc.open_by_key('105cZcOMvRnx4hNA-N7oypGwIXuetCRZJEe1IHf9R45c')
worksheet = workbook.worksheet('機種名変換テーブル')
kisyu_name_convert_df = read_spreadsheet(workbook,worksheet)
kisyu_name_convert_df

serch_kisyu_name_list:list[str] = list(kisyu_name_convert_df['変換前アナスロ機種名'])
serch_kisyu_name_list = list(set(serch_kisyu_name_list))
serch_kisyu_name_list = sorted(serch_kisyu_name_list)
#serch_kisyu_name_list.remove('')
print(len(serch_kisyu_name_list ))
serch_kisyu_name_list.remove('')
serch_kisyu_name_list 

75


['43Lパチスロ北斗の拳ADXR',
 'ANEMONE 交響詩篇エウレカセブン HI‐EVOLUTION',
 'ANEMONE交響詩篇エウレカセブンHIEVOLUTION',
 'Angel Beats!',
 'AngelBeats',
 'Lスマスロ北斗',
 'Lパチスロ北斗の拳ADXR',
 'RYUKYUBEAT30',
 'S Angel beats! XF',
 'S コードギアス',
 'S パチスロコードギアス',
 'Sエウレカセブン HIEVO ZR',
 'SエウレカセブンHIEVOZR',
 'Sコードギアス反逆のルルーシュ',
 'Sコードギアス叛逆のルルーシュKZ',
 'Sハ-ドボイルドXX',
 'Sパチスロ交響詩篇エウレカセブンHI-EVOLZR',
 'Sパチスロ北斗の拳',
 'Sパチスロ幼女戦姫記ZR',
 'Sヱウレカセブン3',
 'S交響詩篇エウレカセブンHI-EVOL',
 'S北斗の拳昇天',
 '‐S盾勇者成り上がり',
 'あの日見た花の名前を僕達はまだ知らない。',
 'けものフレFS',
 'けものフレンズ',
 'この素晴らしい世界に祝福を',
 'この素晴らしい世界に祝福を！',
 'アラジンAクラシック',
 'エウレカセブン',
 'ガメラ',
 'コードギアス反逆のルルーシュ',
 'コードギアス反逆のルルーシュ3',
 'コードギアス反逆のルルーシュZF',
 'スマスロ北斗の拳',
 'チェインクロニクル',
 'ツインエンジェルP',
 'ツインエンジェルPARTY',
 'ディスクUP2',
 'ディスクアUP2',
 'ディスクアップ2',
 'ドリフターズ',
 'ハードボイXX',
 'ハードボイルド',
 'パチスロ ツインエンジェルBREAK',
 'パチスロディスクアップ',
 'パチスロファンシースターオンライン2',
 'パチスロ交響詩篇エウレカセブ',
 'パチスロ猛獣王 王者の咆',
 'パチスロ猛獣王 王者の咆ZZ',
 'ファンタシースターオンライン2',
 'ペルソナ5',
 '七つの大罪',
 '交響詩篇エウレカセブン3 HI-EVOLUTION ZERO',
 '交響詩篇エウレカセブン3HIEVOLUTIONZERO',
 '北斗の拳 天昇',
 '北斗

In [43]:
new_kisyu_name_candidate_list = sorted(list(concat_data_df['機種名'].unique()))
print(len(new_kisyu_name_candidate_list))
kisyu_name_candidate_list = list(set(new_kisyu_name_candidate_list) - set(serch_kisyu_name_list)) 
print(len(kisyu_name_candidate_list))
for x in sorted(kisyu_name_candidate_list):
    print(x)

222
193
2027DB
BIG島唄30
BLACKLAGOONZERObulletMAX
BOOWY
CCエンジェル
CREANewクレアの秘宝伝
FAIRYTAIL2
GANTZ極THESURVIVALGAME
GI優駿倶楽部2
GI優駿倶楽部3
HEYエリートサラリーマン鏡
HEY鏡
KING黄門ちゃま
Lucky海物語
Lパチスロベルセルク無双EV
Lベルセルク無双
OVERLORD絶対支配者光臨II
OVERSLOT「AINZOOALGOWN絶対支配者光臨」
PYRAMIDEYE
Reゼロから始める異世界生活
Reゼロから始める異世界生活ApexVacation
SG1優俊倶楽部2
STEINSGATE
SメイドインアビスEN
S牙狼黄金騎士
TIGERBUNNY
WakeUpGirlsSevenMemories
いろはに愛姫
うしおととら雷槍一閃
うまい棒
えとたま
かまいたちの夜
とある科学の超電磁砲
どんだけ
にゃんこ大戦争BIGBANG
ひぐらしのなく頃に祭2
ひぐらしのなく頃に祭2カケラ遊び編
もっとクレアの秘宝伝女神の歌声と太陽の子供達
もっとチバリヨ25
もっとチバリヨ30
アイドルマスターミリオンライブ
アイムジャグラーEX
アイムジャグラーEXTP
アカメが斬る
アクエリオンALLSTARS
アナザーゴッドハーデス解き放たれし槍撃ver
アナターのオットはーです
アバサー
ウルトラマンタロウ暴君SPEC
エヴァンゲリオンフェスティバル
ガールズケイリンGIフェアリーグランプリ
ガールズパンツァーGこれが私の戦車道です
ガールズパンツァー劇場版
キャッツアイ
キングハナハナ30
ギャグダー
ケロット4
ゴブリンスレイヤー
サラリーマン金太郎MAX
サンダーVライトニング
シャア専用パチスロ逆襲の赤い彗星
シンデレラブレイド4
ジャベリン
スターパルサー
スナイパイ71
スロドル
スーパーハナハナ
スーパーハナハナ30
スーパービンゴギャラクシー
スーパーリオエース
スーパーリノSP
スーパー海物語INJAPAN祭
ゼーガペイン2
ソードアートオンライン
タブータトゥー
ダンまち外伝ソードオラトリア
チバリヨ25
チバリヨ30
デビルメイクライ5
ドンちゃん2
ドンチャン2
ニューゲッターマウス
ニューハナハナゴールド
ニューパルサーDX

In [42]:
worksheet = workbook.worksheet('店舗名変換用テーブル完成版')
tenpo_name_convert_df = read_spreadsheet(workbook,worksheet)
tenpo_name_convert_df = tenpo_name_convert_df.replace({'スロパチID': {'': 99999}})
tenpo_name_convert_df = tenpo_name_convert_df.fillna({'スロパチID': 99999})
tenpo_name_convert_df['スロパチID'] = tenpo_name_convert_df['スロパチID'].astype(int)
tenpo_name_convert_df

,kintone_code,経度,緯度,DMM店舗名,都道府県,市区町村,住所,アナスロ店舗名,ホールナビ店舗名,URL,スロパチID,店舗名／法人名
1,P04961,139.378484,35.4501632,キコーナ海老名店,神奈川県,海老名市,神奈川県海老名市河原口4-5-6,キコーナ海老名店,キコーナ海老名店,https://p-town.dmm.com/shops/kanagawa/4961,506,キコーナ海老名店
2,P00651,139.80573,35.749407,ピーアーク北千住SSS,東京都,足立区,東京都足立区千住旭町3-5駅前会館ビル1Ｆ,ピーアーク北千住SSS,ピーアーク北千住SSS,https://p-town.dmm.com/shops/tokyo/651,513,ピーアーク北千住SSS
3,P01195,140.783334,41.7806327,KEIO湯川店,北海道,函館市,北海道函館市湯川町1-26-12,KEIO湯川店,KEIO湯川店,https://p-town.dmm.com/shops/hokkaido/1195,708,
4,P09568,133.904531,34.6159756,123藤田店,岡山県,岡山市南区,岡山県岡山市南区藤田673-22,123藤田店,123藤田店,https://p-town.dmm.com/shops/okayama/9568,771,123藤田店
5,P09572,133.937835,34.6277287,123浜野店,岡山県,岡山市南区,岡山県岡山市南区浜野2-10-48,123浜野店,123浜野店,https://p-town.dmm.com/shops/okayama/9572,772,
...,...,...,...,...,...,...,...,...,...,...,...,...
2406,,,,,,,,メガフェイス960飯塚,,,99999,MEGAFACE960飯塚
2407,,,,,,,,HYPER ARROW美原店,,,99999,HYPER ARROW 美原店
2408,,,,,,,,パールショップともえセンター南店,,,99999,パールショップともえセンター南店
2409,,,,,,,,123+N布施店,,,99999,123+N布施店


In [ ]:
sammy_sf_list = list(sammy_sf_report_df[sammy_sf_report_df['イベント日'] == '2023/03/31']['店舗名／法人名'].unique())
for x in sorted(sammy_sf_list):
    print(x)

In [7]:
sammy_sf_report_df = pd.read_csv(r'csv\sammy_sf_0331.csv')
#sammy_sf_report_df['shop_id'] = sammy_sf_report_df['shop_id'].astype(int)
sammy_sf_report_df = sammy_sf_report_df.rename(columns={'店舗ID':'スロパチID'})
sammy_sf_report_df = sammy_sf_report_df.fillna({'スロパチID': 99999})
sammy_sf_report_df['スロパチID'] = sammy_sf_report_df['スロパチID'].astype(int)
sammy_sf_report_df

# last_merged_df = pd.merge(sammy_sf_report_df,tenpo_name_convert_df,on='店舗名／法人名',how='left')
# anaslo_serched_tenpo_name_list = list(last_merged_df['アナスロ店舗名'].unique())
# anaslo_serched_tenpo_name_list.pop(6)
# anaslo_serched_tenpo_name_list = sorted(anaslo_serched_tenpo_name_list)
# #anaslo_serched_tenpo_name_list
# last_merged_df

,店舗名／法人名,都道府県,イベント日,提案区分,shop_id,スロパチID
0,123+N羽曳野店,大阪府,2023/03/31,サミーの日,7998.0,8940
1,123+N大阪本店,大阪府,2023/03/31,サミーの日,12564.0,8153
2,123+Ｎ松原店,大阪府,2023/03/31,サミーの日,7948.0,8826
3,123横浜西口店,神奈川県,2023/03/31,サミーの日,12985.0,970
4,123鶴橋店,大阪府,2023/03/31,サミーの日,7344.0,8395
...,...,...,...,...,...,...
63,新！ガーデン北与野,埼玉県,2023/03/31,サミーの日,3312.0,3588
64,仙川UNO,東京都,2023/03/31,サミーの日,885.0,5416
65,中山UNO,神奈川県,2023/03/31,サミーの日,4564.0,4157
66,中山ZoRoN,神奈川県,2023/03/31,サミーの日,12719.0,20615


In [ ]:
for tenpo_name in ['123門真店','123+N大阪本店','レッドロック学芸大学駅前店']:
    display(serched_df[serched_df['店舗名'] ==tenpo_name])
    serched_df[serched_df['店舗名'] ==tenpo_name].to_csv(fr'csv\{tenpo_name}_sammy_kisyu.csv',index=False)

In [50]:

#date_list:list[str] = ['03-30','03-31']
error_kisyu_name_list:list[str] = []
error_tenpo_name_list:list[str] = []
sammy_groupby_serched_df_list = []
for date_str in target_day_list:
    month_str = date_str.split('-')[0]
    #print(month_str)
    #anaslo_serched_tenpo_name_list:list[str] = list(set(kisyu_name_convert_df[f'{month_str}月_店舗名']))
    anaslo_serched_tenpo_name_list = concat_data_df['店舗名'].unique()
    print(len(anaslo_serched_tenpo_name_list))
    sorted(anaslo_serched_tenpo_name_list)
    #anaslo_serched_tenpo_name_list.remove('')
    #print(anaslo_serched_tenpo_name_list)
    csv_file_name  = f'csv\\{date_str}_scraping_anaslo.csv'
    target_date = csv_file_name.split('_')[0].split('\\')[-1]
    data_df =  pd.read_csv(csv_file_name)
    serched_df = pd.DataFrame(index=[], columns=list(data_df.columns))
    display(data_df)
    for serch_tenpo_name in list(data_df['店舗名'].unique()):
        #try:
        #print(serch_tenpo_name)
        extract_tenpo_data_df = data_df[data_df['店舗名'] == serch_tenpo_name]
        print(serch_tenpo_name,len(extract_tenpo_data_df))
        for kisyu_name in serch_kisyu_name_list :
            #print(kisyu_name)
            try:
                serch_kisyu_name_record = extract_tenpo_data_df[extract_tenpo_data_df['機種名'] == kisyu_name]
                if len(serch_kisyu_name_record) == 0:
                    error_kisyu_name_list.append(kisyu_name)# + f'_{date_str}'
                serched_df= pd.concat([serched_df, serch_kisyu_name_record], axis=0)
                #print('成功',kisyu_name)
            except:
                print('失敗',kisyu_name)
                error_kisyu_name_list.append(kisyu_name)
                continue
            #break
        # except:
        #     error_tenpo_name_list.append(serch_tenpo_name)
        #     continue
        #display(extract_tenpo_data_df)
        #break
        #break
    #serch_kisyu_name_list.remove('')
    serched_df['G数'] = serched_df['G数'].astype(int)
    groupby_serched_df = serched_df.groupby('店舗名').mean()#.sort_values('G数',ascending=False)#.reset_index()
    groupby_serched_df['サミー総台数'] = serched_df.groupby(['店舗名']).size()
    new_column_gamesuu_column:str = target_date + '_' + 'サミー全台平均G数'
    groupby_serched_df = groupby_serched_df.rename(columns={'G数':new_column_gamesuu_column})
    groupby_serched_df = groupby_serched_df[[ new_column_gamesuu_column,'サミー総台数']]#groupby_serched_df_list
    sammy_groupby_serched_df_list.append(groupby_serched_df)


100


,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
0,339,123+N和歌山本店,モンキーターンIV,705,102,-185.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-06-03
1,340,123+N和歌山本店,モンキーターンIV,706,965,-641.0,0,1,2.0,1/321.7,1/0.0,1/965.0,1/482.5,2023-06-03
2,341,123+N和歌山本店,モンキーターンIV,707,287,-126.0,0,1,2.0,1/95.7,1/0.0,1/287.0,1/143.5,2023-06-03
3,342,123+N和歌山本店,戦姫絶唱シンフォギア勇気の歌,708,1712,580.0,0,6,21.0,1/63.4,1/0.0,1/285.3,1/81.5,2023-06-03
4,343,123+N和歌山本店,戦姫絶唱シンフォギア勇気の歌,709,1310,-1534.0,0,4,2.0,1/218.3,1/0.0,1/327.5,1/655.0,2023-06-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34938,131,BEAM by HIKARI,沖ドキGOLD,1172,1647,2980.0,25,5,NaN,1/54.9,1/65.9,1/329.4,NaN,2023-06-03
34939,132,BEAM by HIKARI,沖ドキGOLD,1173,3668,1670.0,26,19,NaN,1/81.5,1/141.1,1/193.1,NaN,2023-06-03
34940,133,BEAM by HIKARI,沖ドキGOLD,1175,1686,-646.0,7,5,NaN,1/140.5,1/240.9,1/337.2,NaN,2023-06-03
34941,134,BEAM by HIKARI,沖ドキGOLD,1176,2824,2941.0,28,19,NaN,1/60.1,1/100.9,1/148.6,NaN,2023-06-03


123+N和歌山本店 396
123+N大阪本店 480
123+N布施店 661
123+N羽曳野店 443
123+N松原店 470
123岩岡店 380
123横浜西口店 332
123長吉店 240
123門真店 446
123難波店 376
123鶴橋店 391
BEAM HARA 311
GRAND G-ONE甲賀水口店 434
HYPER ARROW美原店 391
KING BOSS 1000 360
メガコンコルド1020刈谷知立店 425
メガコンコルド1111BLAZE店 481
MGM小山本郷店 360
PIA京急川崎 298
PIA八王子 478
PIA川崎ダイス 334
PIA柏 スロット館 343
PIA横浜モアーズ 400
PIA町田新館 386
スーパーDステーション入間店 289
スーパーDステーション鴻巣店 361
WIN-WIN ONE 273
アスカ狭山店 240
アビバ湘南台店 501
アビバ関内 318
ジュラク王子店 377
アムズガーデン石巻湊店 160
ウエスタン環七南葛西店 206
エクスアリーナ柏 400
エクスアリーナ東京 480
キコーナ吹田店 320
キコーナ守口店 363
キコーナ小田原扇町店 240
キコーナ海老名店 433
グランキコーナ堺店 697
グランキコーナ相模原店 491
グランドオータ瀬谷店 406
スロットサントロペ横須賀中央 290
サンパレス環七大杉店 289
ジャムフレンドクラブ朝霞店 238
スロット123梶取店 235
スーパーライブガーデン行田店 374
テンガイ・与次郎店 400
ニューヨークニューヨーク泉店 152
ニューヨークニューヨーク臨海店 220
パラッツォ三郷中央店 520
パラッツォ鳩ヶ谷店 754
パールショップともえセンター南店 639
パールショップともえ市原店 328
パールショップともえ松戸店 428
ビッグアップル.加古川 424
ビッグアップル．秋葉原店 462
ピーアーク三田 175
ピーアーク千葉駅前 345
ピーアーク相模原 363
ピーアーク相模大野 456
ピーアーク竹ノ塚スタジオ 268
ピーアーク草加 257
プレイランド第一平和 214
マルハンなんば新館 436
マルハンなんば本館 394
マルハンメガシティ羽曳野 254
マル

,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
0,339,123+N和歌山本店,モンキーターンIV,705,0,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-06-02
1,340,123+N和歌山本店,モンキーターンIV,706,0,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-06-02
2,341,123+N和歌山本店,モンキーターンIV,707,844,91.0,0,1,6.0,1/120.6,1/0.0,1/844.0,1/140.7,2023-06-02
3,342,123+N和歌山本店,戦姫絶唱シンフォギア勇気の歌,708,0,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-06-02
4,343,123+N和歌山本店,戦姫絶唱シンフォギア勇気の歌,709,0,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-06-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34958,131,BEAM by HIKARI,沖ドキGOLD,1172,727,1458.0,10,4,NaN,1/51.9,1/72.7,1/181.8,NaN,2023-06-02
34959,132,BEAM by HIKARI,沖ドキGOLD,1173,1877,4039.0,32,5,NaN,1/50.7,1/58.7,1/375.4,NaN,2023-06-02
34960,133,BEAM by HIKARI,沖ドキGOLD,1175,408,-546.0,0,1,NaN,1/408.0,1/0.0,1/408.0,NaN,2023-06-02
34961,134,BEAM by HIKARI,沖ドキGOLD,1176,1697,1169.0,17,3,NaN,1/84.8,1/99.8,1/565.7,NaN,2023-06-02


123+N和歌山本店 396
123+N大阪本店 480
123+N布施店 661
123+N羽曳野店 443
123+N松原店 470
123岩岡店 380
123横浜西口店 332
123長吉店 240
123門真店 446
123難波店 376
123鶴橋店 391
BEAM HARA 311
GRAND G-ONE甲賀水口店 434
HYPER ARROW美原店 391
KING BOSS 1000 360
メガコンコルド1020刈谷知立店 425
メガコンコルド1111BLAZE店 481
MGM小山本郷店 360
PIA京急川崎 298
PIA八王子 478
PIA川崎ダイス 334
PIA柏 スロット館 343
PIA横浜モアーズ 400
PIA町田新館 386
スーパーDステーション入間店 289
スーパーDステーション鴻巣店 361
WIN-WIN ONE 273
アスカ狭山店 240
アビバ湘南台店 501
アビバ関内 318
ジュラク王子店 381
アムズガーデン石巻湊店 160
ウエスタン環七南葛西店 206
エクスアリーナ柏 400
エクスアリーナ東京 480
キコーナ吹田店 320
キコーナ守口店 363
キコーナ小田原扇町店 240
キコーナ海老名店 433
グランキコーナ堺店 697
グランキコーナ相模原店 491
グランドオータ瀬谷店 406
スロットサントロペ横須賀中央 290
サンパレス環七大杉店 289
ジャムフレンドクラブ朝霞店 238
スロット123梶取店 235
スーパーライブガーデン行田店 374
テンガイ・与次郎店 400
ニューヨークニューヨーク泉店 152
ニューヨークニューヨーク臨海店 220
パラッツォ三郷中央店 520
パラッツォ鳩ヶ谷店 754
パールショップともえセンター南店 655
パールショップともえ市原店 328
パールショップともえ松戸店 428
ビッグアップル.加古川 424
ビッグアップル．秋葉原店 462
ピーアーク三田 175
ピーアーク千葉駅前 345
ピーアーク相模原 363
ピーアーク相模大野 456
ピーアーク竹ノ塚スタジオ 268
ピーアーク草加 257
プレイランド第一平和 214
マルハンなんば新館 436
マルハンなんば本館 394
マルハンメガシティ羽曳野 254
マル

In [53]:

error_kisyu_name_list:list[str] = []
error_tenpo_name_list:list[str] = []
tenpo_groupby_serched_df_list = []
for date_str in target_day_list:
    month_str = date_str.split('-')[0]
    print(month_str)
    #anaslo_serched_tenpo_name_list:list[str] = list(set(kisyu_name_convert_df[f'{month_str}月_店舗名']))
    print(len(anaslo_serched_tenpo_name_list))
    sorted(anaslo_serched_tenpo_name_list)
    #anaslo_serched_tenpo_name_list.remove('')
    print(anaslo_serched_tenpo_name_list)
    csv_file_name  = f'csv\\{date_str}_scraping_anaslo.csv'
    target_date = csv_file_name.split('_')[0].split('\\')[-1]
    data_df =  pd.read_csv(csv_file_name)
    serched_df = pd.DataFrame(index=[], columns=list(data_df.columns))
    for serch_tenpo_name in list(data_df['店舗名'].unique()):
        try:
            #print(serch_tenpo_name)
            # try:
            #     shop_id:int = int(slopachi_tenpo_name_and_shopid_df[slopachi_tenpo_name_and_shopid_df['店舗名／法人名'] == serch_tenpo_name].iloc[0,1])
            # except:
            #     print(serch_tenpo_name)
            #     continue
            try:
                #anaslo_tenpo_name = tenpo_name_convert_df[tenpo_name_convert_df['shop_id'] == shop_id].iloc[0,6]
                extract_tenpo_data_df = data_df[data_df['店舗名'] == serch_tenpo_name]
            except Exception as e :
                print(e)
                try:
                    extract_tenpo_data_df = data_df[data_df['店舗名'] == serch_tenpo_name]
                except Exception as e:
                    print('エラー',serch_tenpo_name,e)
                    error_tenpo_name_list.append(serch_tenpo_name)
                    continue
            serched_df= pd.concat([serched_df, extract_tenpo_data_df], axis=0)
        except Exception as e :
            error_tenpo_name_list.append(serch_tenpo_name)
            print(e)
            continue
        #display(extract_tenpo_data_df)
        #break
        #break
    #serch_kisyu_name_list.remove('')
    serched_df['G数'] = serched_df['G数'].astype(int)
    groupby_serched_df = serched_df.groupby('店舗名').mean()#.sort_values('G数',ascending=False)#.reset_index()
    groupby_serched_df['総台数'] = serched_df.groupby(['店舗名']).size()
    new_column_gamesuu_column:str = target_date + '_' + '全台平均G数'
    groupby_serched_df = groupby_serched_df.rename(columns={'G数':new_column_gamesuu_column})
    groupby_serched_df = groupby_serched_df[[ new_column_gamesuu_column,'総台数']]#groupby_serched_df_list
    tenpo_groupby_serched_df_list.append(groupby_serched_df)



2023
100
['123+N和歌山本店' '123+N大阪本店' '123+N布施店' '123+N羽曳野店' '123+N松原店' '123岩岡店'
 '123横浜西口店' '123長吉店' '123門真店' '123難波店' '123鶴橋店' 'BEAM HARA'
 'GRAND G-ONE甲賀水口店' 'HYPER ARROW美原店' 'KING BOSS 1000' 'メガコンコルド1020刈谷知立店'
 'メガコンコルド1111BLAZE店' 'MGM小山本郷店' 'PIA京急川崎' 'PIA八王子' 'PIA川崎ダイス' 'PIA柏 スロット館'
 'PIA横浜モアーズ' 'PIA町田新館' 'スーパーDステーション入間店' 'スーパーDステーション鴻巣店' 'WIN-WIN ONE'
 'アスカ狭山店' 'アビバ湘南台店' 'アビバ関内' 'ジュラク王子店' 'アムズガーデン石巻湊店' 'ウエスタン環七南葛西店'
 'エクスアリーナ柏' 'エクスアリーナ東京' 'キコーナ吹田店' 'キコーナ守口店' 'キコーナ小田原扇町店' 'キコーナ海老名店'
 'グランキコーナ堺店' 'グランキコーナ相模原店' 'グランドオータ瀬谷店' 'スロットサントロペ横須賀中央' 'サンパレス環七大杉店'
 'ジャムフレンドクラブ朝霞店' 'スロット123梶取店' 'スーパーライブガーデン行田店' 'テンガイ・与次郎店'
 'ニューヨークニューヨーク泉店' 'ニューヨークニューヨーク臨海店' 'パラッツォ三郷中央店' 'パラッツォ鳩ヶ谷店'
 'パールショップともえセンター南店' 'パールショップともえ市原店' 'パールショップともえ松戸店' 'ビッグアップル.加古川'
 'ビッグアップル．秋葉原店' 'ピーアーク三田' 'ピーアーク千葉駅前' 'ピーアーク相模原' 'ピーアーク相模大野'
 'ピーアーク竹ノ塚スタジオ' 'ピーアーク草加' 'プレイランド第一平和' 'マルハンなんば新館' 'マルハンなんば本館'
 'マルハンメガシティ羽曳野' 'マルハン上牧店' 'マルハン加島店' 'マルハン堺遠里小野店' 'マルハン大安寺店' 'マルハン布施店'
 'マルハン新世界店' 'マルハン梅田店' 'マルハン橿原北店' 'マルハン草薙アピア店' 'マルハン貝塚店' 'マル

In [54]:
pd.concat(tenpo_groupby_serched_df_list + sammy_groupby_serched_df_list,axis=1)

,2023-06-03_全台平均G数,総台数,2023-06-02_全台平均G数,総台数,2023-06-03_サミー全台平均G数,サミー総台数,2023-06-02_サミー全台平均G数,サミー総台数
店舗名,,,,,,,,
123+N和歌山本店,3600.585859,396,1885.540404,396,5243.347826,92,2480.891304,92
123+N大阪本店,5893.114583,480,4479.102083,480,6570.752381,105,5773.838095,105
123+N布施店,4976.561271,661,3204.630862,661,6316.714286,112,4056.687500,112
123+N松原店,4075.855319,470,2865.078723,470,5749.531646,79,4971.734177,79
123+N羽曳野店,4676.282167,443,3571.216704,443,5960.825581,86,4758.430233,86
...,...,...,...,...,...,...,...,...
楽園沼津店,4208.062678,351,1323.051282,351,5679.826667,75,2324.520000,75
楽園袋井店,2991.806548,336,1464.297619,336,4343.266667,60,2151.100000,60
秋葉原UNO,2933.305699,193,1613.709845,193,4887.250000,40,1804.825000,40


In [55]:
pd.concat(tenpo_groupby_serched_df_list + sammy_groupby_serched_df_list,axis=1).to_csv(r'csv\0603_0605出力1.csv')
#pd.concat(temo_df_list).to_csv('三か月分実施.csv')

In [ ]:
slopachi_tenpo_name_and_shopid_df =  pd.read_csv(r'csv\slopachi_tenpo_name_and_shopid.csv')
slopachi_tenpo_name_and_shopid_df =slopachi_tenpo_name_and_shopid_df[(~slopachi_tenpo_name_and_shopid_df['店舗ID'].isnull()) & (~slopachi_tenpo_name_and_shopid_df['都道府県'].isnull())]
slopachi_tenpo_name_and_shopid_df = slopachi_tenpo_name_and_shopid_df[['記事用店舗名','店舗ID','店舗名／法人名','都道府県']]
slopachi_tenpo_name_and_shopid_df['店舗ID'] = slopachi_tenpo_name_and_shopid_df['店舗ID'].astype(int)
slopachi_tenpo_name_and_shopid_df.drop_duplicates(subset='店舗名／法人名', keep='last', inplace=True)
slopachi_tenpo_name_and_shopid_df

In [50]:
serched_df['日付'] = f"2023-{date}"
serched_df.to_csv(rf'csv\2023-{date}_scraping_anaslo.csv',index=False)

In [66]:
data_df.to_csv(r'csv\2023-03-02_anaslo copy.csv',index=False)

In [ ]:
csv_file_name  = f'csv\\2023-03-03_anaslo copy.csv'
target_date = csv_file_name.split('_')[0].split('\\')[-1]
data_df =  pd.read_csv(csv_file_name)
data_df



====== WebDriver manager ======
Current google-chrome version is 112.0.5615
Get LATEST chromedriver version for 112.0.5615 google-chrome
Driver [C:\Users\slotd\.wdm\drivers\chromedriver\win32\112.0.5615.49\chromedriver.exe] found in cache
C:\Users\slotd\AppData\Local\Temp/ipykernel_50400/1895281330.py:51: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())


NameError: name 'prefecture_name_list' is not defined

In [6]:
sf_past_all_record_df = pd.read_csv(r'csv\sf_past_all_record.csv')
sf_past_all_record_df = sf_past_all_record_df[~sf_past_all_record_df['店舗ID'].isnull()]
sf_past_all_record_df = sf_past_all_record_df[~sf_past_all_record_df.duplicated(subset=['店舗名／法人名'])]
sf_past_all_record_df = sf_past_all_record_df[['店舗名／法人名','店舗ID']]
sf_past_all_record_df['店舗ID'] = sf_past_all_record_df['店舗ID'].astype(int)
sf_past_all_record_df.reset_index(drop=True,inplace=True)
sf_past_all_record_df = sf_past_all_record_df.rename(columns={'店舗ID':'スロパチID'})
sf_past_all_record_df



,店舗名／法人名,スロパチID
0,オゼック羽村,5488
1,ガーデン松戸金ヶ作,3281
2,パーラートモエ,4851
3,スロットスタジアムフォーション,4898
4,TAKARAJIMA荻島店,3723
...,...,...
1517,LOVE,7456
1518,スロット123梶取店,7370
1519,アミューズ岩出,851
1520,メトロヒルズ,7371


In [12]:
parlar_name_parlar_url_dict_df = pd.read_csv(r'csv\tenpo_name_tenpo_url_dict.csv', header=None, names=['店舗名', 'url'])
parlar_name_parlar_url_dict_df
parlar_name_parlar_url_dict = d
ict(zip(parlar_name_parlar_url_dict_df['店舗名'],parlar_name_parlar_url_dict_df['url'] ))
#parlar_name_parlar_url_dict

In [25]:
import csv
dct = slopachi_and_anaslo_tenpo_name_dict
with open(r'csv\slopachi_and_anaslo_tenpo_name_dict.csv', 'w') as f:  
    writer = csv.writer(f)
    for k, v in dct.items():
       writer.writerow([k, v])

In [8]:
for i,tenpo_name in enumerate(tenpo_name_list):
    anaslo_parlar_name_url = parlar_name_parlar_url_dict[tenpo_name]
    anaslo_url_parlarname =urllib.parse.unquote((anaslo_parlar_name_url.split('/')[-2])).replace('-データ一覧','')
    print(i,anaslo_url_parlarname)

NameError: name 'tenpo_name_list' is not defined

In [43]:
list(set(anaslo_scraping_error_tenpo_name_list))

['Dステーション新小岩店',
 'メガフェイス1180座間',
 'グランキコーナ泉佐野店',
 'Dステーション仙台東店',
 'オゼック国分寺北口',
 'スーパーDステーション立川店',
 'MEGA BEAM ASAKURA 999',
 'EVO2']

In [56]:
hand_computed_tenpo_name_list = '''レッドロック学芸大学駅前店
ニューヨークニューヨーク泉店
第一プラザ西浦和店'''.split('\n')

In [63]:
concat_data_df

,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
0,339,123+N和歌山本店,モンキーターンIV,705,0,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-06-02
1,340,123+N和歌山本店,モンキーターンIV,706,0,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-06-02
2,341,123+N和歌山本店,モンキーターンIV,707,844,91.0,0,1,6.0,1/120.6,1/0.0,1/844.0,1/140.7,2023-06-02
3,342,123+N和歌山本店,戦姫絶唱シンフォギア勇気の歌,708,0,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-06-02
4,343,123+N和歌山本店,戦姫絶唱シンフォギア勇気の歌,709,0,0.0,0,0,0.0,1/0.0,1/0.0,1/0.0,1/0.0,2023-06-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34938,131,BEAM by HIKARI,沖ドキGOLD,1172,1647,2980.0,25,5,NaN,1/54.9,1/65.9,1/329.4,NaN,2023-06-03
34939,132,BEAM by HIKARI,沖ドキGOLD,1173,3668,1670.0,26,19,NaN,1/81.5,1/141.1,1/193.1,NaN,2023-06-03
34940,133,BEAM by HIKARI,沖ドキGOLD,1175,1686,-646.0,7,5,NaN,1/140.5,1/240.9,1/337.2,NaN,2023-06-03
34941,134,BEAM by HIKARI,沖ドキGOLD,1176,2824,2941.0,28,19,NaN,1/60.1,1/100.9,1/148.6,NaN,2023-06-03


In [64]:
for tenpo_name in hand_computed_tenpo_name_list:
    display(concat_data_df[concat_data_df['店舗名'] ==tenpo_name])
    concat_data_df[concat_data_df['店舗名'] ==tenpo_name].to_csv(fr'csv\{tenpo_name}_2023-06-03.csv',index=False)

,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
29776,135,レッドロック学芸大学駅前店,犬夜叉,1,0,0.0,0,0,NaN,1/0.0,1/0.0,1/0.0,NaN,2023-06-02
29777,136,レッドロック学芸大学駅前店,バーサスリヴァイズ,2,7291,-558.0,21,23,NaN,1/165.7,1/347.2,1/317.0,NaN,2023-06-02
29778,137,レッドロック学芸大学駅前店,新ハナビ,3,4306,-1020.0,14,5,NaN,1/226.6,1/307.6,1/861.2,NaN,2023-06-02
29779,126,レッドロック学芸大学駅前店,この素晴らしい世界に祝福を,5,230,-54.0,3,1,NaN,1/57.5,1/76.7,1/230.0,NaN,2023-06-02
29780,127,レッドロック学芸大学駅前店,この素晴らしい世界に祝福を,6,1646,-1020.0,20,4,NaN,1/68.6,1/82.3,1/411.5,NaN,2023-06-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29894,64,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,213,2812,-1837.0,14,1,NaN,1/187.5,1/200.9,1/2812.0,NaN,2023-06-03
29895,65,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,215,3215,231.0,18,9,NaN,1/119.1,1/178.6,1/357.2,NaN,2023-06-03
29896,66,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,216,1607,939.0,6,2,NaN,1/200.9,1/267.8,1/803.5,NaN,2023-06-03
29897,67,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,217,1116,1075.0,6,4,NaN,1/111.6,1/186.0,1/279.0,NaN,2023-06-03


,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
18001,107,ニューヨークニューヨーク泉店,沖ドキGOLD30,501,1812,-1000.0,7,5,NaN,1/151.0,1/258.9,1/362.4,NaN,2023-06-02
18002,108,ニューヨークニューヨーク泉店,沖ドキGOLD30,502,2795,-1507.0,11,5,NaN,1/174.7,1/254.1,1/559.0,NaN,2023-06-02
18003,109,ニューヨークニューヨーク泉店,沖ドキGOLD30,503,2330,62.0,14,5,NaN,1/122.6,1/166.4,1/466.0,NaN,2023-06-02
18004,110,ニューヨークニューヨーク泉店,沖ドキGOLD30,504,1867,-1453.0,6,1,NaN,1/266.7,1/311.2,1/1867.0,NaN,2023-06-02
18005,87,ニューヨークニューヨーク泉店,チバリヨ30,505,871,-973.0,1,1,NaN,1/435.5,1/871.0,1/871.0,NaN,2023-06-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18144,116,ニューヨークニューヨーク泉店,ニューパルサーDX3,648,1096,-1160.0,0,2,NaN,1/548.0,1/0.0,1/548.0,NaN,2023-06-03
18145,97,ニューヨークニューヨーク泉店,新ハナビ,649,4713,-120.0,16,13,NaN,1/162.5,1/294.6,1/362.5,NaN,2023-06-03
18146,119,ニューヨークニューヨーク泉店,バーサスリヴァイズ,650,7952,1938.0,32,24,NaN,1/142.0,1/248.5,1/331.3,NaN,2023-06-03
18147,151,ニューヨークニューヨーク泉店,うまい棒,651,1712,387.0,9,0,NaN,1/190.2,1/190.2,1/0.0,NaN,2023-06-03


,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
32087,109,第一プラザ西浦和店,ハッピージャグラーVIII,449,3341,-829.0,11,6,0.0,1/196.5,1/303.7,1/556.8,1/0.0,2023-06-02
32088,110,第一プラザ西浦和店,ハッピージャグラーVIII,450,2335,-19.0,9,6,0.0,1/155.7,1/259.4,1/389.2,1/0.0,2023-06-02
32089,111,第一プラザ西浦和店,ハッピージャグラーVIII,451,4970,280.0,20,18,0.0,1/130.8,1/248.5,1/276.1,1/0.0,2023-06-02
32090,69,第一プラザ西浦和店,ファンキージャグラー2,452,2300,-570.0,7,6,0.0,1/176.9,1/328.6,1/383.3,1/0.0,2023-06-02
32091,70,第一プラザ西浦和店,ファンキージャグラー2,453,2734,-528.0,9,8,0.0,1/160.8,1/303.8,1/341.8,1/0.0,2023-06-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32214,64,第一プラザ西浦和店,ソードアートオンライン,596,2946,-2501.0,10,13,0.0,1/128.1,1/294.6,1/226.6,1/0.0,2023-06-03
32215,65,第一プラザ西浦和店,ソードアートオンライン,597,4316,5101.0,78,23,0.0,1/42.7,1/55.3,1/187.7,1/0.0,2023-06-03
32216,66,第一プラザ西浦和店,ソードアートオンライン,598,4059,3933.0,68,25,0.0,1/43.6,1/59.7,1/162.4,1/0.0,2023-06-03
32217,67,第一プラザ西浦和店,ソードアートオンライン,599,3310,-776.0,29,17,0.0,1/72.0,1/114.1,1/194.7,1/0.0,2023-06-03


In [51]:
hand_comupted_sammy_serched_df = pd.DataFrame(index=[], columns=list(data_df.columns))
for tenpo_name in hand_computed_tenpo_name_list:
    
    display(serched_df[serched_df['店舗名'] ==tenpo_name])
    extract_tenpo_data_df = concat_data_df[concat_data_df['店舗名'] ==tenpo_name]
    for kisyu_name in serch_kisyu_name_list :
        #print(kisyu_name)
        try:
            serch_kisyu_name_record = extract_tenpo_data_df[extract_tenpo_data_df['機種名'] == kisyu_name]
            if len(serch_kisyu_name_record) == 0:
                error_kisyu_name_list.append(kisyu_name)# + f'_{date_str}'
            hand_comupted_sammy_serched_df = pd.concat([hand_comupted_sammy_serched_df, serch_kisyu_name_record], axis=0)
            
            #print('成功',kisyu_name)
        except:
            pass
    hand_comupted_sammy_serched_df.to_csv(fr'csv\hand_computed_sammy_2023-05-03.csv',index=False)

,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
22747,131,レッドロック学芸大学駅前店,犬夜叉,1,0,0.0,0,0,NaN,1/0.0,1/0.0,1/0.0,NaN,2023-05-03
22748,133,レッドロック学芸大学駅前店,バーサスリヴァイズ,2,4418,1448.0,21,12,NaN,1/133.9,1/210.4,1/368.2,NaN,2023-05-03
22749,134,レッドロック学芸大学駅前店,新ハナビ,3,2882,151.0,11,6,NaN,1/169.5,1/262.0,1/480.3,NaN,2023-05-03
22750,129,レッドロック学芸大学駅前店,ハードボイルド,5,3251,-207.0,10,5,NaN,1/216.7,1/325.1,1/650.2,NaN,2023-05-03
22751,130,レッドロック学芸大学駅前店,ハードボイルド,6,1507,-1256.0,2,0,NaN,1/753.5,1/753.5,1/0.0,NaN,2023-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22885,64,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,213,3064,-207.0,12,7,NaN,1/161.3,1/255.3,1/437.7,NaN,2023-05-03
22886,65,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,215,1852,-207.0,13,5,NaN,1/102.9,1/142.5,1/370.4,NaN,2023-05-03
22887,66,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,216,2657,262.0,15,5,NaN,1/132.8,1/177.1,1/531.4,NaN,2023-05-03
22888,67,レッドロック学芸大学駅前店,HEYエリートサラリーマン鏡,217,1850,-2130.0,5,0,NaN,1/370.0,1/370.0,1/0.0,NaN,2023-05-03


,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
16821,38,ZAP PRESTIGE,ハッピージャグラーVIII,385,1617,-706.0,3,5,NaN,1/202.1,1/539.0,1/323.4,NaN,2023-05-03
16822,39,ZAP PRESTIGE,ハッピージャグラーVIII,386,630,-355.0,1,1,NaN,1/315.0,1/630.0,1/630.0,NaN,2023-05-03
16823,40,ZAP PRESTIGE,ハッピージャグラーVIII,387,3484,-1352.0,6,12,NaN,1/193.6,1/580.7,1/290.3,NaN,2023-05-03
16824,41,ZAP PRESTIGE,ハッピージャグラーVIII,388,767,-324.0,2,1,NaN,1/255.7,1/383.5,1/767.0,NaN,2023-05-03
16825,42,ZAP PRESTIGE,ハッピージャグラーVIII,389,2438,-179.0,10,3,NaN,1/187.5,1/243.8,1/812.7,NaN,2023-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16968,67,ZAP PRESTIGE,沖ドキGOLD,536,3882,-1464.0,19,7,NaN,1/149.3,1/204.3,1/554.6,NaN,2023-05-03
16969,68,ZAP PRESTIGE,沖ドキGOLD,537,4084,591.0,26,16,NaN,1/97.2,1/157.1,1/255.2,NaN,2023-05-03
16970,69,ZAP PRESTIGE,沖ドキGOLD,538,5607,-1924.0,27,15,NaN,1/133.5,1/207.7,1/373.8,NaN,2023-05-03
16971,70,ZAP PRESTIGE,沖ドキGOLD,539,4539,6205.0,55,19,NaN,1/61.3,1/82.5,1/238.9,NaN,2023-05-03


,Unnamed: 0,店舗名,機種名,台番号,G数,差枚,BB,RB,ART,合成確率,BB確率,RB確率,ART確率,日付
25239,103,第一プラザ西浦和店,ハッピージャグラーVIII,449,2103,-1309.0,4,3,0.0,1/300.4,1/525.8,1/701.0,1/0.0,2023-05-03
25240,104,第一プラザ西浦和店,ハッピージャグラーVIII,450,8520,1652.0,38,28,0.0,1/129.1,1/224.2,1/304.3,1/0.0,2023-05-03
25241,105,第一プラザ西浦和店,ハッピージャグラーVIII,451,2900,-1993.0,5,4,0.0,1/322.2,1/580.0,1/725.0,1/0.0,2023-05-03
25242,63,第一プラザ西浦和店,ファンキージャグラー2,452,5349,209.0,24,10,0.0,1/157.3,1/222.9,1/534.9,1/0.0,2023-05-03
25243,64,第一プラザ西浦和店,ファンキージャグラー2,453,3769,-1952.0,8,8,0.0,1/235.6,1/471.1,1/471.1,1/0.0,2023-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25386,26,第一プラザ西浦和店,HEYエリートサラリーマン鏡,596,3908,335.0,16,7,0.0,1/169.9,1/244.2,1/558.3,1/0.0,2023-05-03
25387,27,第一プラザ西浦和店,HEYエリートサラリーマン鏡,597,5915,-2497.0,23,11,0.0,1/174.0,1/257.2,1/537.7,1/0.0,2023-05-03
25388,28,第一プラザ西浦和店,HEYエリートサラリーマン鏡,598,2268,-1367.0,12,2,0.0,1/162.0,1/189.0,1/1134.0,1/0.0,2023-05-03
25389,29,第一プラザ西浦和店,HEYエリートサラリーマン鏡,599,2700,-1660.0,11,3,0.0,1/192.9,1/245.5,1/900.0,1/0.0,2023-05-03
